In [4]:
%pip install --upgrade tf-nightly

    100% |████████████████████████████████| 97.2MB 13kB/s 
Requirement already up-to-date: google-pasta>=0.1.6 in /home/jupyter/.local/lib/python3.5/site-packages (from tf-nightly)
    100% |████████████████████████████████| 501kB 2.8MB/s 
Requirement already up-to-date: gast>=0.2.0 in /usr/local/lib/python3.5/dist-packages (from tf-nightly)
Requirement already up-to-date: grpcio>=1.8.6 in /home/jupyter/.local/lib/python3.5/site-packages (from tf-nightly)
    100% |████████████████████████████████| 3.1MB 468kB/s 
Requirement already up-to-date: termcolor>=1.1.0 in /usr/local/lib/python3.5/dist-packages (from tf-nightly)
    100% |████████████████████████████████| 51kB 9.9MB/s 
Requirement already up-to-date: six>=1.10.0 in /usr/local/lib/python3.5/dist-packages (from tf-nightly)
    100% |████████████████████████████████| 51kB 9.4MB/s 
Requirement already up-to-date: protobuf>=3.6.1 in /home/jupyter/.local/lib/python3.5/site-packages (from tf-nightly)
Requirement already up-to-date: wr

In [1]:
import tensorflow as tf
tf.enable_v2_behavior()
import pandas as pd
print(tf.__version__)

inputs = {
    'a' : tf.keras.layers.Input(name='a', shape=(), dtype='float32'),
    'b' : tf.keras.layers.Input(name='b', shape=(), dtype='string'),
}
fc = {
    'a' : tf.feature_column.numeric_column('a'),
    'b' : tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list('b', ['one', 'two'])
    )
}

ds = tf.keras.layers.DenseFeatures(fc.values())(inputs)
output = tf.keras.layers.Dense(1)(ds)
model = tf.keras.models.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='mse')

df = pd.DataFrame.from_dict({
    'a': [3.0, 2.0, 1.0],
    'b': ['one', 'one', 'two'],
})
y =  [3.0, 2.0, 2.0]

model.fit([df['a'], df['b']], y, steps_per_epoch=1)

model.predict({
    'a': tf.convert_to_tensor([3.0]),
    'b': tf.convert_to_tensor(['one']),
})

W0607 17:46:00.692675 140493231068928 deprecation_wrapper.py:118] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/__init__.py:93: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0607 17:46:00.694152 140493231068928 deprecation_wrapper.py:118] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/__init__.py:93: The name tf.AttrValue is deprecated. Please use tf.compat.v1.AttrValue instead.

W0607 17:46:00.695152 140493231068928 deprecation_wrapper.py:118] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/__init__.py:93: The name tf.COMPILER_VERSION is deprecated. Please use tf.version.COMPILER_VERSION instead.

W0607 17:46:00.696009 140493231068928 deprecation_wrapper.py:118] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/__init__.py:93: The name tf.CXX11_ABI_FLAG is deprecated. Please use tf.sysconfig.CXX11_ABI_FLAG instead.

W0607 17:46:00.696871 140493231068928 deprecation_wrapper.py:118

1.14.1-dev20190607


W0607 17:46:01.006426 140493231068928 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:2703: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0607 17:46:01.036107 140493231068928 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:4273: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0607 17:46:01.037275 140493231068928 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/feature_column/f

Train on 1 samples
1/1 [==============================] - 0s 75ms/step - loss: 1.8946


array([[1.3874505]], dtype=float32)

In [2]:
# This doesn't work.
model.save('04_keras/export/simple.h5')
newmodel = tf.keras.models.load_model('04_keras/export/simple.h5')
newmodel.predict({
    'a': tf.convert_to_tensor([3.0]),
    'b': tf.convert_to_tensor(['one']),
})

TypeError: in converted code:
    relative to /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/feature_column:

    feature_column_v2.py:474 call  *
        tensor = column.get_dense_tensor(transformation_cache,
    feature_column_v2.py:4357 get_dense_tensor
        return transformation_cache.get(self, state_manager)
    feature_column_v2.py:2592 get
        if key in self._feature_tensors:

    TypeError: unhashable type: 'list'


In [9]:
# this should work, but doesn't.
import shutil
shutil.rmtree('04_keras/export/savedmodel') #, ignoreErrors=True)
tf.keras.experimental.export_saved_model(model, '04_keras/export/savedmodel')

W0607 18:00:03.962171 140493231068928 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0607 18:00:03.964139 140493231068928 export_utils.py:182] Export includes no default signature!
W0607 18:00:04.196800 140493231068928 export_utils.py:182] Export includes no default signature!
